In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.nn as nn
import torch.nn.functional as F
import sys
sys.path.append("../input/timm-pytorch-image-models/pytorch-image-models-master/")
import timm
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from torchvision import transforms as T
from sklearn.metrics import accuracy_score
!mkdir images
def get_img(path):
    im_bgr = cv2.imread(path)

    im_rgb = im_bgr[:, :, ::-1]
    #print(im_rgb)
    return im_rgb

img = get_img('../input/plants114514/MK/D1/train/Class (1)/R_0Class1 (10).jpg')
im_bgr = cv2.imread("../input/plants114514/MK/D1/train/Class (1)/R_0Class1 (10).jpg")
cv2.imwrite("./images/img.jpg",im_bgr)
plt.imshow(img)
plt.show()

In [ ]:
import os
import math
import time
import random
import shutil
from pathlib import Path
from contextlib import contextmanager
from collections import defaultdict, Counter

import scipy as sp
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

from tqdm.auto import tqdm
from functools import partial

import cv2
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD
import torchvision.models as models
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau

from albumentations import (
    Compose, OneOf, Normalize, Resize, RandomResizedCrop, RandomCrop, HorizontalFlip, VerticalFlip, 
    RandomBrightness, RandomContrast, RandomBrightnessContrast, Rotate, ShiftScaleRotate, Cutout, 
    IAAAdditiveGaussianNoise, Transpose
    )
from albumentations.pytorch import ToTensorV2
from albumentations import ImageOnlyTransform

import timm
from pathlib import Path
import warnings 
warnings.filterwarnings('ignore')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Image preprocessing

In [ ]:

img = get_img('../input/plants114514/MK/D1/train/Class (1)/R_0Class1 (10).jpg')
Trans_pose =Transpose(p=1)
img_out = Trans_pose.apply(img)

Horizontal_Flip =HorizontalFlip(p=1)
img_out_Horizontal_Flip = Horizontal_Flip.apply(img)

VerticalFlip1 =VerticalFlip(p=1)
img_out_VerticalFlip1 = VerticalFlip1.apply(img)


VerticalFlip1 =VerticalFlip(p=1)
img_out_VerticalFlip1 = VerticalFlip1.apply(img)



img_out_ShiftScaleRotate = ShiftScaleRotate(p=1)(image=img)["image"]

Normalize1 = Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            )

img_out_Normalize1 = Normalize1.apply(img)

!mkdir images
plt.imshow(img_out)
plt.show()
plt.savefig("./images/img_out_transpose.png")
plt.imshow(img_out_Horizontal_Flip)
plt.show()
plt.savefig("./images/img_out_Horizontal_Flip.png")

plt.imshow(img_out_VerticalFlip1)
plt.show()
plt.savefig("./images/img_out_VerticalFlip1.png")

plt.imshow(img_out_ShiftScaleRotate)

plt.show()
plt.savefig("./images/img_out_ShiftScaleRotate.png")

plt.imshow(img_out_Normalize1)
plt.show()
plt.savefig("./images/img_out_Normalize1.png")
from pathlib import Path
img_root = Path('./images')
import zipfile
with zipfile.ZipFile('imgs.zip', 'w') as z:
    for img_name in img_root.iterdir():
        z.write(img_name)

Utils

In [ ]:
def get_transforms(mode):
    
    if mode == 'train':
        return Compose([
            Transpose(p=0.5),
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            ShiftScaleRotate(p=0.5),
            Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])

    elif mode == 'val':
        return Compose([
            Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])
    

class PLantDataSet(Dataset):
    
    def __init__(self, images, device, transform=None):
        self.images = images
        self.transform = transform
    
    def __len__(self):
        return self.images.shape[0]
    
    def __getitem__(self, item):
        img = cv2.imread(self.images.iloc[item, 0])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if self.transform is not None:
            img = self.transform(image=img)
        label = self.images.iloc[item, 1]
        img = torch.tensor(img["image"]).float().to(device)
        label = torch.tensor(label).long().to(device)
        return img, label

Models

In [ ]:
def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


'''vgg make layers'''


def make_layers(cfg, batch_norm=False):
    layers = []
    in_channels = 3
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    layers += [nn.Conv2d(in_channels, 512, kernel_size=3, padding=1), nn.BatchNorm2d(512), nn.ReLU(inplace=True)]
    layers += [nn.MaxPool2d(kernel_size=3)]
    return nn.Sequential(*layers)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out







class Flatten(nn.Module):
    
    def __init__(self):
        super().__init__()
    
    def forward(self, x):
        return x.view(x.shape[0], -1)


class VGGNet(nn.Module):
    
    def __init__(self, num_classification, freeze=True):
        super().__init__()
        self.vgg = timm.create_model("vgg16", pretrained=True)
        self.vgg.head = nn.Identity()
        if freeze:
            for param in self.vgg.parameters():
                param.requires_grad = False
        self.classify = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            Flatten(),
            nn.Linear(4096, num_classification))
    
    def forward(self, x):
        x = self.vgg(x)
        return self.classify(x)
    
    
class VGG16Bilinear(nn.Module):
    def __init__(self, num_classification, freeze=False):
        super().__init__()
        vgg = timm.create_model("vgg16", pretrained=True)
        vgg.head = nn.Identity()
        if freeze:
            for param in vgg.parameters():
                param.requires_grad = False
        
        self.cnn = nn.Sequential(vgg, nn.AdaptiveAvgPool2d((1, 1)), nn.Conv2d(4096, 64, 1), nn.ReLU())
        x = torch.rand(1, 3, 256, 256)
        _, c, h, w = self.cnn(x).shape
        
        if freeze:
            for param in self.cnn.parameters():
                param.requires_grad = False
        
        self.classifier = nn.Linear(c ** 2, num_classification)
        
    def forward(self, x):
        x = self.cnn(x)
        b, c, h, w = x.shape
        x = torch.bmm(x.view(b, c, h * w), x.view(b, c, h * w).transpose(1, 2)).view(b, -1)
        x = torch.nn.functional.normalize(torch.sign(x) * torch.sqrt(torch.abs(x) + 1e-10))
        x = self.classifier(x)
        return x

class VGG16Bilinear_impro(nn.Module):
    def __init__(self, num_classification, freeze=False):
        super().__init__()
        vgg = timm.create_model("vgg16", pretrained=True)
        vgg.head = nn.Identity()
        if freeze:
            for param in vgg.parameters():
                param.requires_grad = False
        
        self.cnn = nn.Sequential(vgg, nn.AdaptiveAvgPool2d((1, 1)), nn.Conv2d(4096, 64, 1), nn.ReLU())
        x = torch.rand(1, 3, 256, 256)
        _, c, h, w = self.cnn(x).shape
        
        if freeze:
            for param in self.cnn.parameters():
                param.requires_grad = False
        
        # self.classifier = nn.Linear(c ** 2, num_classification)
        self.class1 = nn.Bilinear(64, 64, 4096)
        self.class2 = nn.Linear(4096, num_classification)

    def forward(self, x):
        x = self.cnn(x)
        b, c, h, w = x.shape
        x_= x.view(x.size(0), -1)
        x_=self.class1(x_, x_)
        x_=self.class2(x_)
        
        # x = torch.bmm(x.view(b, c, h * w), x.view(b, c, h * w).transpose(1, 2)).view(b, -1)
        # x = torch.nn.functional.normalize(torch.sign(x) * torch.sqrt(torch.abs(x) + 1e-10))
        # x = self.classifier(x)
        return x_



class VGG16_Res18_Bilinear(nn.Module):

    def __init__(self, num_classes=3):
        super(VGG16_Res18_Bilinear, self).__init__()

        '''resnet50'''
        # resnet50
        # block = BasicBlock
        # layers = [2, 2, 2, 2]
        # self.inplanes = 64
        # self.features_A = nn.Sequential(
        #     nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
        #               bias=False),
        #     nn.BatchNorm2d(64),
        #     nn.ReLU(inplace=True),
        #     nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
        #     self._make_layer(block, 64, layers[0]),
        #     self._make_layer(block, 128, layers[1], stride=2),
        #     self._make_layer(block, 256, layers[2], stride=2),
        #     self._make_layer(block, 512, layers[3], stride=2),
        #     nn.AvgPool2d(7, stride=1),
        # )
        # for m in self.features_A.modules():
        #     if isinstance(m, nn.Conv2d):
        #         n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
        #         m.weight.data.normal_(0, math.sqrt(2. / n))
        #     elif isinstance(m, nn.BatchNorm2d):
        #         m.weight.data.fill_(1)
        #         m.bias.data.zero_()

        # self.classif_alex = nn.Sequential(
        #     nn.Dropout(),
        #     nn.Linear(256 * 6 * 6, 1024),
        #     nn.ReLU(inplace=True),
        # )

        '''vgg16 bn =True'''
        cfg = [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M']
        self.features_A = make_layers(cfg, batch_norm=True)
        # self.classier_A =  nn.Sequential(
        #     nn.Linear(512 * 3 * 3, 1024),
        #     nn.ReLU(True),
        #     nn.Dropout(),
        #     nn.Linear(1024, 512),
        #     nn.ReLU(True),
        #     nn.Dropout(),
        # )
        self._initialize_weights(self.features_A)
        # self._initialize_weights(self.classier_A)

        '''resnet18'''
        block = BasicBlock
        layers = [2, 2, 2, 2]
        self.inplanes = 64
        self.features_B = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                      bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            self._make_layer(block, 64, layers[0]),
            self._make_layer(block, 128, layers[1], stride=2),
            self._make_layer(block, 256, layers[2], stride=2),
            self._make_layer(block, 512, layers[3], stride=2),
            # nn.AvgPool2d(7, stride=1),
            nn.AdaptiveAvgPool2d((1,1)),
        )
        for m in self.features_B.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

        # self.classifier = nn.Sequential(
        #     nn.Bilinear(1024, 512, 256),
        #     nn.Linear(256, num_classes),
        # )
        self.class1 = nn.Bilinear(2048, 512, 256)
        self.class2 = nn.Linear(256, num_classes)

    '''resnet make layer'''

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    '''vgg init '''

    def _initialize_weights(self, model):
        for m in model.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()

    def forward(self, x):
        x_A = x.clone()
        x_B = x.clone()
        # print x_A.size()
        # print x_B.size()
        x_A = self.features_A(x_A)
        # print x_A.size()
        x_A = x_A.view(x_A.size(0), -1)
        # x_A = self.classier_A(x_A)
        # x_A = self.classif_alex(x_A)
        # print x_A.size()
        # print x_B.size()
        x_B = self.features_B(x_B)
        x_B = x_B.view(x_B.size(0), -1)
        # print x_A.size()
        # print x_B.size()
        # x = self.classifier(x_A, x_B)
        x = self.class1(x_A, x_B)
        # x = torch.nn.functional.normalize(torch.sign(x) * torch.sqrt(torch.abs(x) + 1e-10))
        x = self.class2(x)
        return x



    
    




Data Prepare

In [ ]:
from pathlib import Path    
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

In [ ]:
train_images = []
d1_train_path = Path("../input/plants114514/MK/D1/train")
for d in d1_train_path.iterdir():
    for f in d.glob("*.jpg"):
        train_images.append((str(f.absolute()), d.name))
train_images = pd.DataFrame(train_images, columns=["path", "label"])

test_images = []
d1_test_path = Path("../input/plants114514/MK/D1/test")
for d in d1_test_path.iterdir():
    for f in d.glob("*.jpg"):
        test_images.append((str(f.absolute()), d.name))
test_images = pd.DataFrame(test_images, columns=["path", "label"])

In [ ]:
label_enc = LabelEncoder().fit(train_images.label)
train_images["label"] = label_enc.transform(train_images["label"])
test_images["label"] = label_enc.transform(test_images["label"])

Model Train

In [ ]:
def train_epoch(model, optimizer, loss_fn, data_loader):
    losses = []
    model.train()
    for imgs, labels in data_loader:
        preds = model(imgs)
        loss = loss_fn(preds, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    return np.mean(losses)


def eval_epoch(model, loss_fn, data_loader):
    losses = []
    acces = []
    model.eval()
    with torch.no_grad():
        for imgs, labels in data_loader:
            preds = model(imgs)
            loss = loss_fn(preds, labels)
            pred_labels = torch.argmax(preds, dim=1).cpu().numpy()
            labels = labels.cpu().numpy()
            acc = accuracy_score(pred_labels, labels)
            losses.append(loss.item())
            acces.append(acc)
    return np.mean(losses), np.mean(acces)

In [ ]:
device = "cuda"
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device:",device)
batch_size = 128
lr = 0.0001   ####only vgg16 use 0.003
num_classifications = 44
train_transform = get_transforms("train")
valid_transform = get_transforms("val")
loss_fn = nn.CrossEntropyLoss()
num_epochs = 2   ##实际使用时使用 100个 epoch
#early_stopping = 5
from tensorboardX import SummaryWriter

writer = SummaryWriter('BCNN')

if num_epochs != 100:
    print("当前num_epochs:",num_epochs)
    print("实际使用时请设置num_epochs为",100)
fold = StratifiedKFold(5)
test_set = PLantDataSet(test_images, device, valid_transform)
test_dl = DataLoader(test_set, batch_size=batch_size, shuffle=False)
for cv, (trn_idx, val_idx) in enumerate(fold.split(train_images.index, train_images.label)):
    
    model = VGGNet(num_classifications)
#     model = VGG16Bilinear(num_classifications)
    #model = VGG16Bilinear_impro(num_classifications)

    #model = VGG16_Res18_Bilinear(num_classifications)

    model.to(device)
    optimizer = Adam(filter(lambda x: x.requires_grad, model.parameters()), lr)
    best_acc = -float("inf")
    best_acc_epoch = 0
    
    trn = train_images.iloc[trn_idx].reset_index(drop=True)
    val = train_images.iloc[val_idx].reset_index(drop=True)
    trn_set = PLantDataSet(trn, device, train_transform)
    val_set = PLantDataSet(val, device, valid_transform)
    trn_dl = DataLoader(trn_set, batch_size=batch_size, shuffle=True)
    val_dl = DataLoader(val_set, batch_size=batch_size, shuffle=False)
    
    for epoch in range(num_epochs):
        train_loss = train_epoch(model, optimizer, loss_fn, trn_dl)
        val_loss, val_acc = eval_epoch(model, loss_fn, val_dl)
        if cv == 0:
            writer.add_scalar('Val/Loss', val_loss,epoch)
            writer.add_scalar('Val/Acc',val_acc,epoch)
        
        print(f"CV {cv+1} epoch {epoch} train loss {train_loss:.3f}, val loss {val_loss:.3f} acc {val_acc:.3f}")
        if val_acc > best_acc:
            torch.save(model.state_dict(), f"cv_vgg_{cv+1}_best.pth")
    print(f"stop training, load best model epoch {epoch+1}")
    model.load_state_dict(torch.load(f"cv_vgg_{cv+1}_best.pth"))
    test_loss, test_acc = eval_epoch(model, loss_fn, test_dl)
    print(f"cv {cv+1} test acc {test_acc:.3f}")
    writer.close()  

In [ ]:
# !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip### 只需要运行一次 下载解压后就可以注释了
# !unzip ngrok-stable-linux-amd64.zip
#这个模块最终出现一个链接 点开就是正确率曲线 以及loss曲线
# import os
# import multiprocessing


# pool = multiprocessing.Pool(processes = 10)
# results_of_processes = [pool.apply_async(os.system, args=(cmd, ), callback = None )
#                         for cmd in [
#                         f"tensorboard --logdir ./BCNN/ --host 0.0.0.0 --port 6006 &",
#                         "./ngrok http 6006 &"
#                         ]]
# ! curl -s http://localhost:4040/api/tunnels | python3 -c \
#     "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [ ]:

#####特征图


def draw_CAM(model, img_path, save_path, PATH=None,transform=None, visual_heatmap=False):
    '''
    绘制 Class Activation Map
    :param model: 加载好权重的Pytorch model
    :param img_path: 测试图片路径
    :param save_path: CAM结果保存路径
    :param transform: 输入图像预处理方法
    :param visual_heatmap: 是否可视化原始heatmap（调用matplotlib）
    :return:
    '''
    # 图像加载&预处理
    # img = Image.open(img_path).convert('RGB')
    img = cv2.imread(img_path)

    # img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

    if transform:
        img = transform(image=img)
    print(model)
    if PATH is not None:
        model.load_state_dict(torch.load(PATH))
    img = img["image"].unsqueeze(0)
 
    # 获取模型输出的feature/score
    model.eval()

    # output = model(img)

    # print(model.cnn._modules['0'].features._modules['6'])
    n=0
    l=0
    for ddd in model.children():###VGG

        if n==8:
            break
        # for ddds in ddd:
            

        #     if n==8:
        #         break
            for ggg in ddds.features:
                n+=1
                if n==8:
                    break
                img=ggg(img)
                print()
#     for ddd in model.children():###BCNN

#         if n==8:
#             break
#         for ddds in ddd:
            

#             if n==8:
#                 break
#             for ggg in ddds.features:
#                 n+=1
#                 if n==8:
#                     break
#                 img=ggg(img)
#                 print()
    features=img
    # features = model.cnn._modules['0'].features._modules['6'](img)
    # output = model(img)
    # b, c, h, w = x.shape
    # features = torch.bmm(x.view(b, c, h * w), x.view(b, c, h * w).transpose(1, 2))
    # features_vec=features.view(b, -1)
    # features_vec = torch.nn.functional.normalize(torch.sign(features_vec) * torch.sqrt(torch.abs(features_vec) + 1e-10))
    # output = model.classifier(features_vec)
    
    heatmap = features[0][0].detach().numpy()

 
    img = cv2.imread(img_path)  # 用cv2加载原始图像
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))  # 将热力图的大小调整为与原始图像相同
    heatmap = np.uint8(255 * heatmap)  # 将热力图转换为RGB格式
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)  # 将热力图应用于原始图像
    superimposed_img = heatmap * 0.4 + img  # 这里的0.4是热力图强度因子

    cv2.imwrite(save_path, superimposed_img)  # 将图像保存到硬盘
    plt.imshow(img)
    plt.show()
valid_transform = get_transforms("val")
img_path = "../input/plants114514/MK/D1/train/Class (8)/R_270Class8 (4).jpg"
save_path="./heat_map_vgg_d1.png"
PATH = "./cv_vgg_1_best.pth"
PATH = None

device = "cuda"
model = VGGNet(44)
# model.to(device)
draw_CAM(model, img_path, save_path, PATH,transform=valid_transform, visual_heatmap=True)